# Cluster and HII-regions Multi <a class="tocSkip">

the aim of this notebook is to combine the HII-region and cluster catalogues.
   
This notebook useses multiple galaxies at once.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column
from pnlf.plot import quick_plot

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)


# first we need to specify the path to the raw data
basedir = Path('..')
data_ext = Path('a:') #basedir / 'data' / 'raw' 

sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])
sample_table['power_index'] = 2.3
sample_table['power_index'][sample_table['AO'].mask]=2.8

## Read in data

In [ ]:
hst_sample      = set(['NGC0628','NGC1365','NGC1433', 'NGC1566', 'NGC3351', 'NGC3627', 'NGC4535'])
astrosat_sample = set([x.stem.split('_')[0] for x in (data_ext/'Astrosat').iterdir() if x.is_file() and x.suffix=='.fits'])
muse_sample     = set(sample_table['name'])
complete_sample = hst_sample & astrosat_sample & muse_sample

### The nebulae catalogue

In [ ]:
scalepc = 32

# the original catalogue from Francesco
with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
    
nebulae.rename_columns(['cen_x','cen_y','cen_ra','cen_dec','region_area',
                          'EBV','EBV_ERR','SkyCoord'],
                         ['x_neb','y_neb','ra_neb','dec_neb','area_neb',
                          'EBV_balmer','EBV_balmer_err','SkyCoord_neb'])
    
# some additional properties 
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_dig.fits') as hdul:
    dig = Table(hdul[1].data)
dig['dig/hii'] = dig['dig_median'] / dig['hii_median']

with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_fuv.fits') as hdul:
    fuv = Table(hdul[1].data)
    
with fits.open(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_eq.fits') as hdul:
    eq_width = Table(hdul[1].data)
    
nebulae = join(nebulae,fuv,keys=['gal_name','region_ID'])
nebulae = join(nebulae,eq_width,keys=['gal_name','region_ID'])
nebulae = join(nebulae,dig,keys=['gal_name','region_ID'])

# this will rais a few errors that we just ignore
with np.errstate(divide='ignore',invalid='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    SIII = nebulae['SIII6312_FLUX_CORR']+nebulae['SIII9068_FLUX_CORR']
    nebulae['[SIII]/[SII]'][SII>0] = SIII[SII>0]/SII[SII>0]
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX_CORR']/nebulae['FUV_FLUX_CORR']
    
    
# the nebulae catalogue with additional information
folder = basedir/'data'/'map_nebulae_association'

lst = []
for file in folder.glob(f'*{scalepc}pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    print(f'reading {gal_name}')
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    name=gal_name.lower()
    lst.append(tbl)
nebulae_tmp = vstack(lst)

nebulae = join(nebulae,nebulae_tmp,keys=['gal_name','region_ID'],join_type='outer')

### The association catalogue

In [ ]:
# those files hold the merged association catalogues
with fits.open(basedir/'data'/'interim'/f'phangshst_associations_nuv_ws{scalepc}pc_v1p1.fits') as hdul:
    associations = Table(hdul[1].data)
    
associations['SkyCoord'] = SkyCoord(associations['reg_ra']*u.degree,associations['reg_dec']*u.degree)
associations.rename_columns(['reg_ra','reg_dec','reg_x','reg_y',
                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Mass_MinChiSq','reg_dolflux_Ebv_MinChiSq',
                             'reg_dolflux_Age_MinChiSq_err','reg_dolflux_Mass_MinChiSq_err','reg_dolflux_Ebv_MinChiSq_err',
                             'SkyCoord'],
                            ['ra_asc','dec_asc','x_asc','y_asc','age','mass',
                             'EBV_stars','age_err','mass_err','EBV_stars_err','SkyCoord_asc'])

# Halpha measured in the association masks
with fits.open(basedir/'data'/'interim'/f'phangshst_associations_nuv_ws32pc_v1p1_Halpha.fits') as hdul:
    assoc_Halpha = Table(hdul[1].data)

# the association catalogue matched with the nebuale catalogue
folder = basedir/'data'/'map_nebulae_association'

lst = []
for file in folder.glob(f'*{scalepc}pc_associations.fits'):
    gal_name = file.stem.split('_')[0]
    print(f'reading {gal_name}')
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
assoc_tmp = vstack(lst)


# combine both catalogues
associations = join(associations,assoc_tmp,keys=['gal_name','assoc_ID'])
print(f'{len(associations)} associations in final catalogue')

criteria = np.abs(associations['age']-associations['age_16'])>associations['age_err']
criteria |= np.abs(associations['age']-associations['age_64'])>associations['age_err']

associations['uniform_age'] = ~criteria

### The joined catalogue

In [ ]:
# we can also recreate the table from the association an nebulae catalogues
catalogue = join(nebulae[~nebulae['assoc_ID'].mask],associations,keys=['gal_name','assoc_ID','region_ID'])
#catalogue['SkyCoord_asc'] = SkyCoord(catalogue['ra_asc'],catalogue['dec_asc'])
#catalogue['SkyCoord_neb'] = SkyCoord(catalogue['ra_neb'],catalogue['dec_neb'])
catalogue['HA/FUV'] = catalogue['HA6562_FLUX_CORR']/catalogue['FUV_FLUX_CORR']

# add dig and calculate galactic radius
catalogue['galactic_radius'] = np.nan
for gal_name in np.unique(catalogue['gal_name']):
    centre = sample_table.loc[gal_name]['SkyCoord']
    catalogue['galactic_radius'][catalogue['gal_name']==gal_name] = catalogue[catalogue['gal_name']==gal_name]['SkyCoord_neb'].separation(centre).to(u.arcmin)

print(f'{len(catalogue)} objects in final catalogue')

In [ ]:
# catalogue with Halpha based on association mask
catalogue = join(associations,assoc_Halpha,keys=['gal_name','assoc_ID'])
catalogue['dig/hii'] = catalogue['dig_median'] / catalogue['hii_median']

In [ ]:
print(f'associations: {len(associations)}')
print(f'nebulae: {len(nebulae[np.isin(nebulae["gal_name"],list(hst_sample))])}')
print(f'match: {len(catalogue)}')
print(f'contained: {len(catalogue[catalogue["overlap"]=="contained"])}')

### Table to showcase the sample

In [ ]:
astrosat_sample = [x.stem.split('_')[0] for x in (data_ext/'Astrosat').iterdir() if x.is_file() and x.suffix=='.fits']
muse_sample     = sample_table['name']
hst_sample      = associations['gal_name']
sitelle_sample  = ['NGC0628','NGC2835','NGC3351','NGC3627','NGC4535']

t = Table({
    'Name':muse_sample,
    'MUSE':np.isin(muse_sample,muse_sample),
    'HST':np.isin(muse_sample,hst_sample),
    'Astrosat':np.isin(muse_sample,astrosat_sample),
    'Sitelle':np.isin(muse_sample,sitelle_sample)}
     )

for col in t.columns[1:]:
    t[col] = ['\checkmark' if x else '' for x in t[col] ]
ascii.write(t,sys.stdout, Writer = ascii.Latex)

## Search for correlations

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [0.5,10.5]

criteria = (catalogue['mass']>1e3)
criteria &= catalogue['uniform_age']
#criteria &= (catalogue['overlap']=='contained')
tmp = catalogue[criteria]
print(len(tmp))

x,mean,std = bin_stat(tmp['age'],tmp['eq_width'],xlim)

fig,ax=plt.subplots(figsize=(single_column,single_column))

ax.scatter(tmp['age'],tmp['eq_width'])
ax.errorbar(x,mean,yerr=std,color='black')

ax.set(xlim=xlim,ylim=[0,200])
plt.show()

### Plot the sample (cutouts)

plot all objects in the merged catalogues

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from cluster.plot import single_cutout


criteria = (catalogue['mass']>1e3) 
#criteria &= ~np.isnan(catalogue['HA/FUV'])
criteria &= (catalogue['overlap'] == 'contained')
criteria &= (catalogue['age'] > 10)
tmp = catalogue[criteria]
tmp = tmp

print(f'{name}: {len(tmp)} objects')

size=10*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'all_galaxies_{scalepc}_cutouts'

    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'{gal_name.lower()}_uvis_f275w_exp_drc_sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                # nebulae mask
                filename = data_ext / 'MUSE_DR2.1' / 'Nebulae catalogue' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'HST',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')

            
            ax = next(axes_iter)
            ax = single_cutout(ax,
                             position = row['SkyCoord_neb'],
                             image = F275,
                             mask1 = nebulae_mask,
                             mask2 = associations_mask,
                             label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}",
                             size  = 4*u.arcsecond)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

three color composit with CO emission

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from reproject import reproject_interp
from skimage.measure import find_contours
from pnlf.plot import create_RGB

'''
criteria = (catalogue['mass']>1e3) 
criteria &= (catalogue['overlap'] == 'contained')
#criteria &= (catalogue['age'] > 10)
tmp = catalogue[criteria]
'''
print(f'{len(tmp)} objects')

size=5*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'all_galaxies_{scalepc}_cutouts_rgb_neg_fesc'

width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for j in range(Npages):
        print(f'working on page {j+1} of {Npages}')

        sub_sample = tmp[j*Npage:(j+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                print(f'reading files for {gal_name}')
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'{gal_name.lower()}_uvis_f275w_exp_drc_sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                filename = data_ext / 'MUSE_DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
    
                # nebulae mask
                filename = data_ext / 'MUSE_DR2.1' / 'Nebulae catalogue' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'HST',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')
            
                with fits.open(data_ext/'ALMAv4p0'/f'{gal_name.lower()}_12m+7m+tp_co21_broad_tpeak.fits') as hdul:
                    CO = NDData(data=hdul[0].data,
                                meta=hdul[0].header,
                                wcs=WCS(hdul[0].header))
            
            ax = next(axes_iter)
            
            position = row['SkyCoord_neb']
            
            label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}"

            cutout_F275 = Cutout2D(F275.data,position,size=size,wcs=F275.wcs)
            norm = simple_norm(cutout_F275.data,stretch='linear',clip=False,percent=99.9)

            cutout_CO, _  = reproject_interp(CO,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    
            cutout_Halpha, _  = reproject_interp(Halpha,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    

            rgb = create_RGB(cutout_CO,cutout_Halpha,cutout_F275.data,
                             percentile=[98,98,99.8],weights=[0.7,0.6,1])
            #ax.imshow(cutout_image.data,origin='lower',norm=norm,cmap=plt.cm.gray_r)
            ax.imshow(rgb,origin='lower')

            # plot the nebulae catalogue
            cutout_mask, _  = reproject_interp(nebulae_mask,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape,order='nearest-neighbor')    
            region_ID = np.unique(cutout_mask[~np.isnan(cutout_mask)])

            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_mask)
                blank_mask[cutout_mask==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='tab:green',lw=0.8,label='HII-region')

            # 32 pc
            cutout_32 = Cutout2D(associations_mask.data,position,size=size,wcs=associations_mask.wcs)
            region_ID = np.unique(cutout_32.data[~np.isnan(cutout_32.data)])
            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_32.data)
                blank_mask[cutout_32.data==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='blue',lw=0.8,label='32pc assoc.')
            t = ax.text(0.06,0.87,label, transform=ax.transAxes,color='black',fontsize=8)
            t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
            ax.set_xticks([])
            ax.set_yticks([])
            
        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if j == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

### FUV vs ionization

In [ ]:
from scipy.stats import binned_statistic, pearsonr, spearmanr

sample = set(astrosat_sample) & set(muse_sample)
filename = basedir/'reports'/'all_objects_HaFUV_over_SII'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 4
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
    
#vmin,vmax = np.min(catalogue['HA6562_FLUX_CORR']),np.max(catalogue['HA6562_FLUX_CORR'])
vmin,vmax = 1e-16,1e-14
# loop over the galaxies we want to plot
for name in sorted(sample):  
    
    tmp = nebulae[(nebulae['gal_name']==name)]
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    tmp = tmp[tmp['FUV_FLUX_CORR']>3*tmp['FUV_FLUX_CORR_ERR']]
    tmp = tmp[tmp['SII6716_FLUX_CORR']>3*tmp['SII6716_FLUX_CORR_ERR']]
    tmp = tmp[tmp['SIII9068_FLUX_CORR']>3*tmp['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(tmp['[SIII]/[SII]'],tmp['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(tmp)} objects')

    sc = ax.scatter(tmp['[SIII]/[SII]'],tmp['HA/FUV'],
               c=1e-20*tmp['HA6562_FLUX_CORR'],vmin=vmin,vmax=vmax,
               cmap=plt.cm.plasma,
               norm=mpl.colors.LogNorm(),
               s=1,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.75,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.62,0.05,f'{len(tmp):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,4e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    
    if i==0:
        #ax.remove()
        ax.axis('off')
        cbar = fig.colorbar(sc, ax=ax,
                            label=r'$\mathrm{H}\alpha$ / (erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$)',
                            orientation='horizontal',
                           )
    else:
        ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')


#plt.savefig(filename.with_suffix('.png'),dpi=600)
#plt.savefig(filename.with_suffix('.pdf'),dpi=600)

plt.show()


### Age and Ha/FUV

In [ ]:
from cluster.auxiliary import bin_stat

criteria = (catalogue['mass']>1e3) 
criteria &= ~np.isnan(catalogue['HA/FUV'])
#criteria &= (catalogue['overlap'] == 'contained')
#criteria &= (catalogue['neighbors'] == 0)
tmp = catalogue[criteria]
print(f'sample contains {len(tmp)} objects')


xlim = [0.5,10.5]
fig,ax=plt.subplots(figsize=(6,4))
sc = ax.scatter(tmp['age'],tmp['HA/FUV'],s=10,alpha=0.8,c=tmp['dig/hii'],vmin=0,vmax=1)

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel=r'H$\alpha$ / FUV',xlim=xlim,ylim=[0,100])

fig.colorbar(sc,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/f'all_galaxies_HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(8,3))

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='contained')]
ax1.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='partial')]
sc=ax2.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

ax1.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')
ax2.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.93, 0.11, 0.02, 0.84])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr')

plt.show()

seperate subplot for each galaxy

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky
from scipy.stats import binned_statistic, pearsonr, spearmanr

# '[SIII]/[SII]' , 'HA/FUV', 'AGE_MINCHISQ', 'AGE_BAYES'
x_name, y_name = 'age', 'HA/FUV'
xlim = [0.5,10.5]
bins = 10
max_sep = 1*u.arcsec

#sample = set(np.unique(nebulae['gal_name'])) & hst_sample
sample = np.unique(catalogue['gal_name'])[:-1]

filename = basedir/'reports'/'all_objects_age_over_SII.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 3
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    criteria = (catalogue['gal_name']==name) 
    criteria &= (catalogue['mass']>1e3) 
    #criteria &= ~np.isnan(catalogue['HA/FUV'])
    criteria &= (catalogue['overlap'] == 'contained')
    #criteria &= (catalogue['neighbors'] == 0)
    
    tmp = catalogue[criteria]
    print(f'{name}: {len(tmp)} objects')
    
    mean, bin_edges, binnumber = binned_statistic(tmp[x_name],
                                                  tmp[y_name],
                                                  statistic='mean',
                                                  bins=bins,
                                                  range=xlim)
    std, _, _ = binned_statistic(tmp[x_name],
                                  tmp[y_name],
                                  statistic='std',
                                  bins=bins,
                                  range=xlim)

    ax.scatter(tmp[x_name],tmp[y_name],color='tab:blue',s=1)
    # plot the standard divation with yerr=std
    ax.errorbar((bin_edges[1:]+bin_edges[:-1])/2,mean,fmt='-')
    ax.text(0.65,0.85,f'{name}', transform=ax.transAxes,fontsize=7)

    ax.set(xlim=xlim)
    if i==nrows-1:
        ax.set_xlabel(f'{x_name.replace("_"," ")}')
    if j==0:
        ax.set_ylabel(f'{y_name.replace("_"," ")}')

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky

# '[SIII]/[SII]' , 'HA/FUV'
x_name, y_name, z_name = '[SIII]/[SII]', 'HA/FUV', 'AGE_MINCHISQ'
xlim = [0.5,10.5]
bins = 10
max_sep = 2*u.arcsec

sample = muse_sample & hst_sample & astrosat_sample

filename = basedir/'reports'/'all_objects_FUV_over_SII_with_age.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 2
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # it makes a different if we match the clusters to the nebulae or the other way around
    catalogcoord = clusters[clusters['gal_name']==name].copy()
    matchcoord   = nebulae[nebulae['gal_name']==name].copy()

    idx, sep, _ = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])

    catalogue = matchcoord.copy()

    for col in catalogcoord.columns:
        if col in catalogue.columns:
            catalogue[f'{col}2'] = catalogcoord[idx][col]
        else:
            catalogue[col] = catalogcoord[idx][col]

    catalogue['sep'] = sep
    catalogue = catalogue[sep.__lt__(max_sep)]
    catalogue = catalogue[~np.isnan(catalogue[x_name]) & ~np.isnan(catalogue[y_name]) & (catalogue['AGE_MINCHISQ']<10)]
    print(f'{name}: {len(catalogue)} objects in joined catalogue')

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    #catalogue = catalogue[catalogue['FUV_FLUX_CORR']>3*catalogue['FUV_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SII6716_FLUX_CORR']>3*catalogue['SII6716_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SIII9068_FLUX_CORR']>3*catalogue['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(catalogue)} objects')

    sc = ax.scatter(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'],
                    c=catalogue[z_name],vmin=0, vmax=10,cmap=plt.cm.RdBu_r,
                    s=3,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.7,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.55,0.05,f'{len(catalogue):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,2e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel('[SIII]/[SII]')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

fig.colorbar(sc,ax=axes.ravel().tolist(),label=f'{z_name.replace("_"," ")}')
        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()


### age histograms

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

tmp = associations[(associations['mass']>1e3) & (associations['age']<10)]

ages_con = tmp[tmp['overlap']=='contained']['age']
ages_par = tmp[tmp['overlap']=='partial']['age']
ages_iso = tmp[tmp['overlap']=='isolated']['age']

print(f'ages: con={np.mean(ages_con):.2f}, par={np.mean(ages_par):.2f}, iso={np.mean(ages_iso):.2f}')

ax1.hist(ages_con,bins=bins,histtype='step',label='contained')
ax2.hist(ages_par,bins=bins,histtype='step',label='partially')
ax3.hist(ages_iso,bins=bins,histtype='step',label='isolated')

ax1.set_title(f'contained ({np.nanmean(ages_con):.2f} Myr)')
ax2.set_title(f'partially ({np.nanmean(ages_par):.2f} Myr)')
ax3.set_title(f'isolated ({np.nanmean(ages_iso):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,600],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'all_galaxies_age_hist_contained.pdf',dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky

tmp = associations[(associations['mass']>1e3) & (associations['age']<10)]
idx,sep,_=match_coordinates_sky(tmp['SkyCoord'],nebulae['SkyCoord'])

ages1 = tmp[(sep<0.4*u.arcsec)]['age']
ages2 = tmp[(sep>0.4*u.arcsec) & (sep<0.8*u.arcsec)]['age']
ages3 = tmp[(sep>0.8*u.arcsec)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'$s<0.4"$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'$0.4"<s<0.8"$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'$0.8"<s$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,600],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'all_galaxies_age_hist_sep.pdf',dpi=600)

plt.show()

In [ ]:
tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['age']<10)]

p1,p2=np.nanpercentile(tmp['HA/FUV'],[33,66])

ages1 = tmp[tmp['HA/FUV']>p2]['age']
ages2 = tmp[(tmp['HA/FUV']>p1) & (tmp['HA/FUV']<p2)]['age']
ages3 = tmp[(tmp['HA/FUV']<p1)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'first percentile in Ha/FUV'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'second percentile in Ha/FUV' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'third percentile in Ha/FUV'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,200],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'all_galaxies_age_hist_HaFUV.pdf',dpi=600)

plt.show()

### EBV

we expect 
$$
E(B-V)_{balmer} = 2 \cdot E(B-V)_{stars}
$$


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

criteria = (catalogue['mass']>1e3) #& (catalogue['overlap']=='contained') & (catalogue['age']<10)
tmp = catalogue[criteria]
print(f'sample contains {len(tmp)} objects')

cmap = plt.cm.get_cmap('viridis',5)
fig = plt.figure(figsize=(single_column,single_column/1.1))
ax = fig.add_subplot()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="10%", pad=0.2,)

xlim = [0,10]
sc=ax.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=cmap)
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

#plt.savefig(basedir/'reports'/f'all_galaxies_{scalepc}_EBV_Balmer_vs_Stars.pdf',dpi=600)
plt.show()

In [ ]:
from scipy.stats import spearmanr

age_bins = np.array([0,2,4,6,8,10])

for low,high in zip(age_bins[:-1],age_bins[1:]):
    t = tmp[(tmp['age']>low) & (tmp['age']<high)]
    r,p = spearmanr(t['EBV_stars'],t['EBV_balmer'])
    print(f'{low} to {high} Myr: rho={r:.2f}, {len(t)} objects')


### Corner Plot

In [ ]:
from cluster.plot import corner

tmp = catalogue[(catalogue['overlap']=='contained') & (catalogue['mass']>1e3) & catalogue['uniform_age']]
print(f'sample contains {len(tmp)} objects')

filename = basedir/'reports'/f'all_galaxies_corner.pdf'
columns  = ['age','HA/FUV','eq_width','met_scal','logq_D91','EBV_stars','EBV_balmer','dig_median']
limits   = {'age':(0,10),'eq_width':(0,100),'HA/FUV':(0,50),'HA/NUV':(0,20),'met_scal':(8.4,8.7),'logq_D91':(6,8)}

corner(tmp,columns,limits,nbins=5,filename=filename,vmin=1000,vmax=1e6,figsize=(18,18))

In [ ]:
with open(basedir/'data'/'map_nebulae_association'/f'{name}_{scalepc}pc_nebulae.yml') as f:
    nebulae_dict = yaml.load(f,Loader=yaml.SafeLoader)
with open(basedir/'data'/'map_nebulae_association'/f'{name}_{scalepc}pc_associations.yml') as f:
    associations_dict = yaml.load(f,Loader=yaml.SafeLoader)

## Compare all with Starburst99

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster = Cluster(stellar_model='GENEVAv40',metallicity=0.014)

for each association, calculate the predicted number of ionizing photons based on the mass and the age of the association.

The observed number of ionizing photos is calulcated based on the conversion factor from Niederhofer+2016 (or Kennicutt+98)

$$
Q(\mathrm{H}^0) = 7.31\cdot 10^{11} L(\mathrm{H}\alpha)
$$

we only use a subsample (high mass, contained, speparated etc.)

In [ ]:
from tqdm import tqdm

catalogue['Qpredicted'] = np.nan
HI_rate = cluster.quanta['HI_rate'].value
time = cluster.quanta['Time']
for row in tqdm(catalogue):
    idx = np.argmin(np.abs(time-row['age']*u.Myr))
    row['Qpredicted'] = HI_rate[idx] * row['mass'] / cluster.mass
    
catalogue['distance'] = np.nan
for gal_name in catalogue['gal_name']:
    distance = Distance(distmod=sample_table.loc[gal_name]['(m-M)'])
    catalogue['distance'][catalogue['gal_name']==gal_name] = distance
    
catalogue['L(Ha)'] = (catalogue['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*(catalogue['distance']*u.Mpc)**2).to(u.erg/u.s)
catalogue['Qobserved'] = 7.31e11*catalogue['L(Ha)']/u.erg
fesc = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc

print(f'fesc={np.nanmean(fesc[fesc>0]):.2f} (from {np.sum(fesc>0)} objects)')
print(f"{np.sum(fesc<0)} of {len(catalogue)} ({np.sum(fesc<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
# for objects with negative fesc we redo the analysis with age-age_err
Qpredict_new = []
for row in tqdm(catalogue):
    if row['fesc']<0:
        idx = np.argmin(np.abs(time-(row['age']-row['age_err'])*u.Myr))
        row['Qpredicted'] = ( HI_rate[idx] * row['mass'] / cluster.mass )

fesc = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc

print(f'fesc={np.nanmean(fesc[fesc>0]):.2f} (from {np.sum(fesc>0)} objects)')
print(f"{np.sum(fesc<0)} of {len(catalogue)} ({np.sum(fesc<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
criteria  = (catalogue['mass']>1e3) 
#criteria &= (catalogue['age']<=20) 
#criteria &= (catalogue['uniform_age']) 
criteria &= (catalogue['overlap']=='contained') 
#criteria &= fesc>0
tmp = catalogue[criteria].copy()

print(f'fesc={np.nanmean(tmp[tmp["fesc"]>0]["fesc"]):.2f} (from {np.sum(criteria)} objects)')
print(f"{np.sum(tmp['fesc']<0)} of {len(tmp)} ({np.sum(tmp['fesc']<0)/len(tmp)*100:.1f}%) regions have negative fesc")

### Plot escape fraction

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,0.9*single_column))

print(f'fesc={np.mean(fesc[fesc>0]):.2f}')

Qpredicted = np.logspace(48,52)
cmap = plt.cm.get_cmap('copper',6)
lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved = Qpredicted*(1-f)
    ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)

sc=ax.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['dig/hii'],cmap=cmap,vmin=0.,vmax=1.,s=2,zorder=2)
ax.legend()
fig.colorbar(sc,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
#fig.colorbar(sc,label='$\log_{10} q$ (from Diaz+91)')

ax.set(xlabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ predicted (SB99)',
       ylabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ observed (MUSE)',
       xlim=[48,52],ylim=[48,52])

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 
ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 

plt.savefig(basedir/'reports'/f'escape_fraction.pdf',dpi=600)
plt.show()

In [ ]:
fig,((ax1,ax2),(ax3,ax4),(ax5,ax6)) =plt.subplots(ncols=2,nrows=3,figsize=(two_column,1.2*two_column))

print(f'fesc={np.mean(fesc[fesc>0]):.2f}')

Qpredicted = np.logspace(48,52)

lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved = Qpredicted*(1-f)
    for ax in (ax1,ax2,ax3,ax4,ax5,ax6):
        ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)

cmap = plt.cm.get_cmap('plasma',6)
sc1=ax1.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['logq_D91'],cmap=cmap,vmin=6,vmax=8,s=2,zorder=2)
cmap = plt.cm.get_cmap('copper',6)
sc2=ax2.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['dig/hii'],cmap=cmap,vmin=0,vmax=1,s=2,zorder=2)
cmap = plt.cm.get_cmap('viridis',6)
sc3=ax3.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['EBV_stars'],cmap=cmap,vmin=0,vmax=0.8,s=2,zorder=2)
cmap = plt.cm.get_cmap('cividis',6)
sc4=ax4.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['age'],cmap=cmap,vmin=0,vmax=10,s=2,zorder=2)
cmap = plt.cm.get_cmap('ocean',6)
sc5=ax5.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=np.log10(tmp['mass']),cmap=cmap,vmin=3,vmax=5.3,s=2,zorder=2)
cmap = plt.cm.get_cmap('magma',6)
sc6=ax6.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['galactic_radius'],cmap=cmap,vmin=0,vmax=2,s=2,zorder=2)


ax1.legend()
fig.colorbar(sc1,ax=ax1,label=r'$\log q$ (from Diaz 91)')
fig.colorbar(sc2,ax=ax2,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
fig.colorbar(sc3,ax=ax3,label=r'$E(B-V)_\mathrm{stars}$')
fig.colorbar(sc4,ax=ax4,label=r'age / Myr')
fig.colorbar(sc5,ax=ax5,label=r'$\log_{10} M/\mathrm{M}_\odot$')
fig.colorbar(sc6,ax=ax6,label=r'galactic radius / arcmin')

for ax in (ax5,ax6):
    ax.set(xlabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ predicted')
for ax in (ax1,ax3,ax5):
    ax.set(ylabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ observed')

for ax in (ax1,ax2,ax3,ax4,ax5,ax6):
    ax.set(xlim=[48,52],ylim=[48,52])
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 
    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 
plt.tight_layout()
plt.savefig(basedir/'reports'/f'escape_fraction.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,single_column))

ax.scatter(tmp['fesc'],tmp['dig/hii'])
ax.set(xlim=[0,1])
plt.show()

### mass vs observed Halpha

In [ ]:
# calculate Ha on a grid of different ages/masses with Starburst99
age_grid = np.array([2,6,10,14,18])*u.Myr
mass_grid = np.logspace(2,7,10)
ionizing_photons = np.zeros((len(age_grid),len(mass_grid)))
for i,mass in enumerate(mass_grid):
    scaled_cluster = cluster.scale(mass)
    for j,age in enumerate(age_grid):
        idx = np.argmin(np.abs(scaled_cluster.quanta['Time']-age))
        ionizing_photons[j,i] = scaled_cluster.quanta['HI_rate'][idx].value
distance = 10*u.Mpc
LHa = ionizing_photons / 7.31e11 * u.erg 
FHa = LHa / (1e-20*u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)

In [ ]:
nrows, ncols = 2,4

cmap = plt.cm.get_cmap('copper', 5)
fig=plt.figure(figsize=(two_column,nrows/ncols*two_column))
axes = []
for i,gal_name in enumerate(np.unique(tmp['gal_name'])):
    ax = fig.add_subplot(nrows,ncols,i+1)
    axes.append(ax)
    sub = tmp[tmp['gal_name']==gal_name]
    
    distance = sub['distance'][0]*u.Mpc
    LHa = ionizing_photons / 7.31e11 * u.erg 
    FHa = LHa / (u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)
    
    sc=ax.scatter(sub['mass'],1e-20*sub['HA6562_FLUX_CORR'],s=3,
               c=sub['age'],cmap=cmap,vmin=0,vmax=20,zorder=2)
    
    # plot the theoretical liens
    for j,age in enumerate(age_grid):
        color = cmap(age/20/u.Myr)
        ax.plot(mass_grid,FHa[j,:],label=age,color=color)
    
    ax.set(xlim=[1e3,1e6],ylim=[5e-17,5e-12],xscale='log',yscale='log')
    ax.text(0.05,0.9,f'{gal_name}', transform=ax.transAxes,fontsize=7)

    if i%ncols==0:
        ax.set(ylabel=r'$F(\mathrm{H}\alpha)$ / erg s$^{-1}$ cm$^{-2}$')
    else:
        ax.set_yticklabels([])
    if i//ncols==nrows-1:
        ax.set(xlabel=r'mass / M$_\odot$')
    else:
        ax.set_xticklabels([])

# Create the legend
h,l = axes[0].get_legend_handles_labels()
fig.legend(h,l,
           ncol=5,
           loc="upper center",   # Position of legend
           borderaxespad=0.2,    # Small spacing around legend box
           )
plt.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.93, 0.11, 0.02, 0.84])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr',ticks=age_grid)

plt.savefig(basedir/'reports'/'mass_vs_Halpha.pdf',dpi=600)
plt.show()

### BPT

In [ ]:
def BPT_diagram(R3,N2,S2,O1,label='',filename=None,**kwargs):
    '''create a BPT diagram
    
    Parameters 
    ----------
    
    R3 : array
        log10([OIII]/Hbeta)
        
    N2 : array
        log10([NII]/Halpha)
        
    S2 : array
        log10([SII]/Halpha)
        
    O1 : array
        log10([OI]/Halpha)
        
    Other Parameters
    ----------------
    
    **kwargs : passed to plt.scatter()
    
    '''
    
    fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3))
        
    x = np.linspace(-2.5,2.5,200)
    
    # ---- N2 vs R3 plot ----
    ax1.scatter(N2,R3,**kwargs)
    ax1.plot(x[x<0.47],0.61/(x[x<0.47]-0.47)+1.19,color='black',ls='--',label='Kewley+2001')
    ax1.plot(x[x<-0.032],0.359/(x[x<-0.032]+0.032)+1.083,color='black',ls='-',label='Law+2021')
    #ax1.legend() 
    
    ax1.set(xlim=[-1.5,0.5],ylim=[-1.5,1],
            xlabel=r'$\log ( [\mathrm{N}\,\textsc{ii}] / \mathrm{H}\alpha )$',
            ylabel=r'$\log ([\mathrm{O}\,\textsc{iii}] / \mathrm{H}\beta )$')
    
    # ---- S2 vs R3 plot ----    
    ax2.scatter(S2,R3,**kwargs)
    ax2.plot(x[x<0.32],0.72/(x[x<0.32]-0.32)+1.3,color='black',ls='--',label='Kewley+2001')
    ax2.plot(x[x>-0.33],1.89*x[x>-0.33]+0.76,color='black',ls='--',label='Kewley+2001')
    ax2.plot(x[x<0.198],0.41/(x[x<0.198]-0.198)+1.164,color='black',ls='-',label='Law+2021')

    ax2.set(xlim=[-1.5,0.5],ylim=[-1.5,1],
            xlabel=r'$\log ( [\mathrm{S}\,\textsc{ii}] / \mathrm{H}\alpha )$')

    # ---- O1 vs R3 plot ----
    sc = ax3.scatter(O1,R3,**kwargs)
    ax3.plot(x[x<-0.53],0.73/(x[x<-0.53]+0.53)+1.33,color='black',ls='--',label='Kewley+2001')
    ax3.plot(x,0.612/(x+0.36)+1.179,color='black',ls='-',label='Law+2021')
    
    ax3.set(xlim=[-2.5,-0.5],ylim=[-1.5,1],
            xlabel=r'$\log ( [\mathrm{O}\,\textsc{i}] / \mathrm{H}\alpha )$')

    for ax in (ax1,ax2,ax3):
        ax.grid()
        
    plt.tight_layout()
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.95, 0.2, 0.02, 0.72])
    fig.colorbar(sc,cax=cbar_ax,label=label)
    
    if filename:
        plt.savefig(filename,dpi=600)
        
    plt.show()


BPT_diagram(R3=np.log10(tmp['OIII5006_FLUX_CORR']/tmp['HB4861_FLUX_CORR']),
            N2=np.log10(tmp['NII6583_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),
            S2=np.log10((tmp['SII6716_FLUX_CORR']+tmp['SII6730_FLUX_CORR'])/tmp['HA6562_FLUX_CORR']),
            O1=np.log10(tmp['OI6300_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),
            c=tmp['fesc'],vmin=0,vmax=1,cmap=plt.cm.get_cmap('copper', 5),s=2,
            label=r'$f_\mathrm{esc}$',filename=basedir/'reports'/'BPT.pdf')

In [ ]:
colors = ['#500093','#8B07F3','#B42003','#D65800','#F2BB00']
cmap = mpl.colors.LinearSegmentedColormap('FB',[mpl.colors.to_rgb(x) for x in colors])

plt.scatter([1,2,3,4],[1,2,3,4],c=[0,5,3,4],cmap=cmap)
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))

logbins = np.logspace(3,7,20)
sub = associations[associations['overlap']=='isolated']
ax.hist(sub['mass'],bins=logbins,label='isolated',alpha=0.6)

sub = associations[associations['overlap']=='partial']
ax.hist(sub['mass'],bins=logbins,label='partial',alpha=0.6)

sub = associations[associations['overlap']=='contained']
ax.hist(sub['mass'],bins=logbins,label='contained',alpha=0.6)

ax.set(xlabel='mass / Msun',xscale='log')
ax.legend()
plt.show()

## Plot regions (MUSE & HST over WFI)

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region

sample = set([x.stem.split('_')[0].upper() for x in (data_ext/'HST'/'white_light').iterdir()])
sample = muse_sample
ncols = 5
nrows = int(np.ceil(len(sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(sample):
    
    print(name)
    
    if name in hst_sample:
        with fits.open(data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits') as hdul:
            hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
            hst_whitelight.data[hst_whitelight.data==0] = np.nan
        
    filename = data_ext / 'MUSE_DR2' / 'MUSEDAP' / f'{name}_MAPS.fits'
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))
        
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        

    reg_muse_pix, reg_muse_sky = find_sky_region(Halpha.mask.astype(int),wcs=Halpha.wcs)
    if name in hst_sample:
        reg_hst_pix, reg_hst_sky = find_sky_region(hst_whitelight.mask.astype(int),wcs=hst_whitelight.wcs)
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=8*u.arcmin,wcs=WFI.wcs)
    
    # project from muse to hst coordinates
    reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
    if name in hst_sample:
        reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')

    reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE',lw=0.5)
    if name in hst_sample:
        reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST',lw=0.5)
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects.pdf',dpi=600)
plt.savefig(basedir/'reports'/'all_objects.png',dpi=600)

plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue_file = basedir/'..'/'PNLF'/'data'/'catalogues'/f'{name}_nebulae.txt'
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    catalogue=catalogue[catalogue['type']=='PN']
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:red',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_PN.pdf',dpi=600)
plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue = filter_table(nebulae,gal_name=name)
    #catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:
        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:blue',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_nebulae.pdf',dpi=600)
plt.show()

## SITELLE

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,4))

for name,ax in zip(['NGC0628','NGC2835','NGC3351'],[ax1,ax2,ax3]):
    with fits.open(basedir/'..'/'sitelle'/'data_v2p1'/'maps'/f'{name}_OII_map.fits') as hdul:
        OII = hdul['OII3726_FLUX'].data
        OII_header = hdul['OII3726_FLUX'].header
    
    norm = simple_norm(OII,clip=False,percent=98)
    ax.imshow(OII,norm=norm,origin='lower',cmap=plt.cm.Greys)
    ax.set_title(name)
    ax.axis('off')
    
plt.show()   

use [OII] line to calculate strong line and direct abundances

In [ ]:
from cluster.metallicity import diagnostic_line_ratios

with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/'interim'/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)

nebulae_with_OII = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
nebulae_with_OII = nebulae_with_OII[np.isin(nebulae_with_OII['gal_name'],['NGC0628','NGC2835','NGC3351','NGC4535'])]
nebulae_with_OII = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>0]

line_ratios = diagnostic_line_ratios(nebulae_with_OII)

compare with Figure 8 in Pilyugin+2016 (looks good if R2*=1.4)

In [ ]:
from cluster.metallicity import strong_line_metallicity_R, strong_line_metallicity_S

subsample = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>10*nebulae_with_OII['OII3726_FLUX_CORR_ERR']].copy()
print(f'{len(subsample)} objects in sample')

# looks a lot better with 1.4*R2
subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_S'],subsample['OH_R'],s=4,c=tab10[0])

ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H)$_\mathrm{S}$',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_S']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H)$_\mathrm{S}-$log(O/H)$_\mathrm{R}$')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

with direct method. This requires electron temperature and density. They have to be measured in an itterative process

In [ ]:
from cluster.metallicity import electron_density_sulfur,\
                                electron_temperature_oxygen, electron_temperature_nitrogen,\
                                electron_temperature_sulfur, oxygen_abundance_direct
   
criteria = (line_ratios['OII7319_FLUX_CORR']>7*line_ratios['OII7319_FLUX_CORR_ERR']) & (line_ratios['OII3726_FLUX_CORR']>10*line_ratios['OII3726_FLUX_CORR_ERR'])
subsample = line_ratios[criteria].copy()

subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])
    
# initial guess for the temperature
subsample['t(NII)'] = electron_temperature_nitrogen(subsample['RN2'])
subsample['t(SIII)'] = electron_temperature_sulfur(subsample['RS3'])
subsample['n(SII)']  = electron_density_sulfur(subsample['RS2'],subsample['t(NII)'])

for x in range(10):
    subsample['t(OII)'] = electron_temperature_oxygen(subsample['RO2'],subsample['n(SII)'])
    subsample['n(SII)'] = electron_density_sulfur(subsample['RS2'],subsample['t(OII)'])
    print(np.nanmean(subsample['n(SII)']))

subsample['OH_direct'] = oxygen_abundance_direct(subsample['R2'],subsample['R3'],subsample['t(OII)'],subsample['n(SII)'])


In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_direct'],subsample['OH_R'],s=4,c=tab10[0])
ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H) direct',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_direct']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H) direct$-$log(O/H)$_\mathrm{R}$')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

compare with Figure 7 in Perez-Montero+2017

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))

ax.scatter(np.log10(subsample['R23']),subsample['OH_direct'],s=4,c=subsample['logq_D91'])
ax.set(xlim=[-0.4,1.4],ylim=[7.,9.0],
       xlabel='log R23',
       ylabel='12+log(O/H) direct')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

ax.plot([0.5,1.5],[0.5,1.5],color='black')
ax.scatter(subsample['t(OII)'],subsample['t(NII)'],s=4,c=tab10[0])
ax.set(xlim=[0.5,1.5],ylim=[0.5,1.5],
       xlabel='t([OII])',
       ylabel='t([NII])')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

## Playground

In [ ]:
def measure_dig(data,mask,label,position,factor=1,max_iter=10,size=32,plot=False):
    '''measure the diffuse ionized gas around an HII-region'''
    
    cutout_mask = Cutout2D(mask.data,position,size=(size,size),mode='partial',fill_value=np.nan)
    cutout_data = Cutout2D(data.data,position,size=(size,size),mode='partial',fill_value=np.nan)
    
    area_mask  = np.sum(cutout_mask.data==label)
    input_mask = cutout_mask.data==label
    
    n_iter = 0
    while True:
        n_iter+=1
        boundaries = find_boundaries(input_mask,mode='outer')
        input_mask |=boundaries
        area_boundary = np.sum(input_mask & np.isnan(cutout_mask.data)) 
        if area_boundary > factor*area_mask or n_iter>max_iter: break
            
    if plot:
        fig,ax=plt.subplots(figsize=(5,5))
        ax.imshow(cutout_mask.data,origin='lower')
        mask = np.zeros((*cutout_mask.shape,4))
        mask[input_mask & np.isnan(cutout_mask.data),:] = (1,0,0,0.5)
        ax.imshow(mask,origin='lower')
        plt.show()
        
    #if np.sum(boundaries & np.isnan(cutout_mask.data))==0:
    #    print(f'no boundaries for {label}')
    dig = cutout_data.data[input_mask & np.isnan(cutout_mask.data)]

    return np.median(dig),np.mean(dig),np.sum(dig)



## Old and young populations

In [ ]:
from regions import read_ds9

tmp = catalogue[catalogue['gal_name']=='NGC1365']

# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'NGC1365_uvis_f275w_exp_drc_sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))

associations,associations_mask = read_associations(folder=data_ext/'HST',target='NGC1365',scalepc=32)
reg_young, reg_old = read_ds9(basedir/'data'/'tmp'/'young_old.reg')

# the catalogue
young_sample = tmp[reg_young.contains(tmp['SkyCoord_asc'],wcs=associations_mask.wcs)]
young_sample = young_sample[young_sample['age']<20]
old_sample = tmp[reg_old.contains(tmp['SkyCoord_asc'],wcs=associations_mask.wcs)]
old_sample = old_sample[old_sample['age']>20]

In [ ]:
# the maps
young = np.isin(associations_mask.data,associations[associations['age']<10]['assoc_ID'])
old   = np.isin(associations_mask.data,associations[associations['age']>10]['assoc_ID'])

young = young.astype(float)
young[young==0] = np.nan

old = old.astype(float)
old[old==0] = np.nan

In [ ]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=F275.wcs)

norm = simple_norm(F275.data,clip=False,percent=99)
ax.imshow(F275.data,norm=norm,origin='lower',cmap=plt.cm.Greys,alpha=0.5)
ax.imshow(old,vmin=0,vmax=1,cmap=plt.cm.Reds,alpha=0.8)
ax.imshow(young,vmin=0,vmax=1,cmap=plt.cm.Blues,alpha=0.8)

reg_young_pix = reg_young.to_pixel(F275.wcs)
reg_old_pix = reg_old.to_pixel(F275.wcs)

ax.add_artist(reg_young_pix.as_artist())
ax.add_artist(reg_old_pix.as_artist())


ax.set(xlim=[2000,5000],ylim=[3000,5000])
plt.show()

In [ ]:
xlim = [0,10]


fig,ax=plt.subplots(figsize=(single_column,single_column))

sc = ax.scatter(young_sample['EBV_stars'],young_sample['EBV_balmer'],color='g',label='upper arm')
sc = ax.scatter(old_sample['EBV_stars'],old_sample['EBV_balmer'],color='m',label='lower arm')
ax.legend()
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
#fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

plt.show()